This is the code to generate Multi-omics data for smoothing purpose.

In [1]:
import logging
import anndata as ad
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import scanpy as sc
import sys
import torch
import os
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances


/usr/local/lib/python3.6/dist-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
input_mod1 = ad.read_h5ad('../../fig2/data/PBMC_10X_GEX.h5ad')
input_mod2 = ad.read_h5ad('../../fig2/data/PBMC_10X_ATAC.peak.h5ad')

used_cts = {
    "CD4 Naive", "CD4 TCM", "CD4 TEM", "CD8 Naive", "CD8 TEM_1", "CD8 TEM_2",
    "CD14 Mono", "CD16 Mono", "Memory B", "Naive B"
}  # To match cell types covered in PC Hi-C

input_mod1 = input_mod1[[item in used_cts for item in input_mod1.obs["seurat_annotations"]],:]
input_mod2 = input_mod2[[item in used_cts for item in input_mod2.obs["seurat_annotations"]],:]

input_mod1.obs['cell_type'] = input_mod1.obs['seurat_annotations'].copy()
input_mod2.obs['cell_type'] = input_mod2.obs['seurat_annotations'].copy()


Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


In [7]:
merge = pd.read_csv('../embed/SiaNN_embed.pbmc.txt')
out_x = merge.iloc[0:input_mod1.n_obs,1:].to_numpy().astype('float32')
out_x = out_x.copy(order='C')
out_y = merge.iloc[input_mod1.n_obs:,1:].to_numpy().astype('float32')
out_y = out_y.copy(order='C')


In [8]:
input_mod1.obsm['SiaNN'] = out_x
input_mod2.obsm['SiaNN'] = out_y

In [11]:
input_mod1.write_h5ad('10XMultiome_GEX.PBMC.h5ad')
input_mod2.write_h5ad('10XMultiome_ATAC.PBMC.h5ad')
